In [0]:
!pip install -U -q pyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import os
os.listdir()
os.chdir("drive/My Drive/Colab Notebooks/Text-Generation/PyTorch/")

In [0]:
import argparse
import torch
import data

In [0]:
# parser = argparse.ArgumentParser(description='PyTorch Wikitext-2 Language Model')

# # Model parameters.
# parser.add_argument('--data', type=str, default='./data/wikitext-2',
#                     help='location of the data corpus')
# parser.add_argument('--checkpoint', type=str, default='./model.pt',
#                     help='model checkpoint to use')
# parser.add_argument('--outf', type=str, default='generated.txt',
#                     help='output file for generated text')
# parser.add_argument('--words', type=int, default='1000',
#                     help='number of words to generate')
# parser.add_argument('--seed', type=int, default=1111,
#                     help='random seed')
# parser.add_argument('--cuda', action='store_true',
#                     help='use CUDA')
# parser.add_argument('--temperature', type=float, default=1.0,
#                     help='temperature - higher will increase diversity')
# parser.add_argument('--log-interval', type=int, default=100,
#                     help='reporting interval')
# args = parser.parse_args()

args = {}
args["data"] = "./data/paul_graham/"
args["checkpoint"] = "./model3.pt"
args["outf"] = "generated3.txt"
args["words"] = 1000
args["seed"] = 1111
args["cuda"] = True
args["temperature"] = 1.0
args["log_interval"] = 100

In [0]:
# Set the random seed manually for reproducibility.
torch.manual_seed(args["seed"])
if torch.cuda.is_available():
    if not args["cuda"]:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if args["cuda"] else "cpu")

In [7]:
if args["temperature"] < 1e-3:
    parser.error("--temperature has to be greater or equal 1e-3")

with open(args["checkpoint"], 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

RNNModel(
  (drop): Dropout(p=0.65, inplace=False)
  (encoder): Embedding(32929, 1500)
  (rnn): LSTM(1500, 1500, num_layers=2, dropout=0.65)
  (decoder): Linear(in_features=1500, out_features=32929, bias=True)
)

In [8]:
corpus = data.Corpus(args["data"])
ntokens = len(corpus.dictionary)

is_transformer_model = hasattr(model, 'model_type') and model.model_type == 'Transformer'
if not is_transformer_model:
    hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with open(args["outf"], 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(args["words"]):
            if is_transformer_model:
                output = model(input, False)
                word_weights = output[-1].squeeze().div(args["temperature"]).exp().cpu()
                word_idx = torch.multinomial(word_weights, 1)[0]
                word_tensor = torch.Tensor([[word_idx]]).long().to(device)
                input = torch.cat([input, word_tensor], 0)
            else:
                output, hidden = model(input, hidden)
                word_weights = output.squeeze().div(args["temperature"]).exp().cpu()
                word_idx = torch.multinomial(word_weights, 1)[0]
                input.fill_(word_idx)

            word = corpus.dictionary.idx2word[word_idx]

            outf.write(word + ('\n' if i % 20 == 19 else ' '))

            if i % args["log_interval"] == 0:
                print('| Generated {}/{} words'.format(i, args["words"]))


| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words
